In [ ]:
#Примечание: т.к. работаю в jupyter в ноутбуки приходится заново добавлять классы моделей и датасетов, насколько я знаю из ipynb импортировать 
#в другой нельзя, но также пользовался этим чтобы вносить небольшие правки например количество логируемой моделью информации в homework_experiment

#1.1
# Модифицируйте существующую линейную регрессию:
# - Добавьте L1 и L2 регуляризацию
# - Добавьте early stopping
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from utils import mse, log_epoch


class LinearRegression(nn.Module):
    def __init__(self, in_features):
        super().__init__()
        self.linear = nn.Linear(in_features, 1)

    def forward(self, x):
        return self.linear(x)


def start_train(dataset, in_features=1):
    dataloader = DataLoader(dataset, batch_size=32, shuffle=True)
    print(f'Размер датасета: {len(dataset)}')
    print(f'Количество батчей: {len(dataloader)}')

    # Создаём модель, функцию потерь и оптимизатор
    model = LinearRegression(in_features)
    criterion = nn.MSELoss()
    optimizer = optim.SGD(model.parameters(), lr=0.1)

    # Параметры регуляризации
    l_lambda = 0.01  # коэффициент для регуляризации
    regularization = 'l2'  # тип регуляризации l1, l2 или None

    # Параметры early stopping
    patience = 5  # количество эпох без улучшения перед остановкой
    best_loss = float('inf')
    epochs_without_improvement = 0

    # Обучаем модель
    epochs = 100
    for epoch in range(1, epochs + 1):
        total_loss = 0

        for i, (batch_X, batch_y) in enumerate(dataloader):
            optimizer.zero_grad()
            y_pred = model(batch_X)

            # Основная функция потерь
            loss = criterion(y_pred, batch_y)

            # L2
            if regularization == 'l2':
                l2_reg = torch.tensor(0.)
                for param in model.parameters():
                    l2_reg += torch.norm(param, p=2) ** 2
                loss += l_lambda * l2_reg
            # L1
            elif regularization == 'l1':
                l1_reg = torch.tensor(0.)
                for param in model.parameters():
                    l1_reg += torch.norm(param, p=1)
                loss += l_lambda * l1_reg

            loss.backward()
            optimizer.step()

            total_loss += loss.item()

        avg_loss = total_loss / (i + 1)

        # Early stopping
        if avg_loss < best_loss:
            best_loss = avg_loss
            epochs_without_improvement = 0
            # Сохраняем лучшую модель
            torch.save(model.state_dict(), 'models/best_linreg_torch.pth')
        else:
            epochs_without_improvement += 1
            if epochs_without_improvement >= patience:
                print(f'Early stopping на эпохе {epoch}')
                break

        if epoch % 10 == 0:
            log_epoch(epoch, avg_loss)

def evaluate():
    best_model = LinearRegression(in_features=1)
    best_model.load_state_dict(torch.load('models/best_linreg_torch.pth'))
    best_model.eval()

Добавил L1 и L2 регуляризацию по формулам с лекций, также early stopping с терпением по умолчанию = 5

# 1.2
# не догадался как самостоятельно реализовать многоклассовую классификацию, прошу прощения :(